In [1]:
!zipline ingest -b quandl

[2024-09-07T18:52:37+0000-INFO][zipline.data.bundles.core]
 Ingesting quandl
[2024-09-07T18:52:37+0000-INFO][zipline.data.bundles.quandl]


Aborted!


In [2]:
import zipline.api as zapi
from zipline import run_algorithm
import pandas as pd
import numpy as np
from datetime import datetime
from platypus import NSGAII, Problem, Real

# Alpha factor functions (same as before)
def rank(series):
    return series.rank(pct=True)

def ts_rank(series, window):
    return series.rolling(window).apply(lambda x: rank(x).iloc[-1])

def ts_argmax(series, window):
    return series.rolling(window).apply(np.argmax) + 1

def correlation(x, y, window):
    return x.rolling(window).corr(y)

def alpha_1(data):
    returns = data['close'].pct_change()
    cond = np.where(returns < 0, data['close'].rolling(20).std(), data['close'])
    return rank(ts_argmax(np.power(cond, 2), 5)) - 0.5

def alpha_2(data):
    return -1 * correlation(rank(np.log(data['volume']).diff(2)), rank((data['close'] - data['open']) / data['open']), 6)

def alpha_3(data):
    return -1 * correlation(rank(data['open']), rank(data['volume']), 10)

def alpha_4(data):
    return -1 * ts_rank(rank(data['low']), 9)

def alpha_5(data):
    vwap = (data['close'] * data['volume']).rolling(10).sum() / data['volume'].rolling(10).sum()
    return rank(data['open'] - vwap) * (-1 * abs(rank(data['close'] - vwap)))

def alpha_6(data):
    return -1 * correlation(data['open'], data['volume'], 10)

# Define your optimization class
class AlphaOptimization(Problem):
    def __init__(self, data, num_alphas=6):
        super(AlphaOptimization, self).__init__(num_alphas, 2)  # Sharpe and volatility
        self.data = data
        self.types[:] = [Real(0, 1) for _ in range(num_alphas)]  # Weights for each alpha

    def evaluate(self, solution):
        weights = solution.variables
        portfolio_returns = sum(weights[i] * self.data[f'alpha_{i+1}'] for i in range(len(weights)))
        sharpe_ratio = portfolio_returns.mean() / portfolio_returns.std()
        volatility = portfolio_returns.std()
        solution.objectives[:] = [-sharpe_ratio, volatility]  # Minimize volatility, maximize Sharpe

# Backtest Logic: Zipline algorithm
def initialize(context):
    context.alphas = []
    context.num_alphas = 6
    context.opt_weights = [1 / context.num_alphas] * context.num_alphas  # Equal weighting as starting point

    # Schedule functions (e.g., rebalance the portfolio every month)
    zapi.schedule_function(rebalance, zapi.date_rules.month_start(), zapi.time_rules.market_open())

# Define the rebalance logic for your strategy
def rebalance(context, data):
    # Combine alpha factors with the optimized weights
    alpha_combination = sum(context.opt_weights[i] * context.alphas[i] for i in range(len(context.alphas)))

    # Decide whether to buy or sell based on the alpha combination
    if alpha_combination > 0:
        zapi.order_target_percent(context.asset, 1.0)  # Full allocation
    elif alpha_combination < 0:
        zapi.order_target_percent(context.asset, -1.0)  # Short the asset

# Define the before_trading_start logic to load alphas
def before_trading_start(context, data):
    # Calculate alpha factors on the historical data
    data = data.history(context.asset, fields=['open', 'high', 'low', 'close', 'volume'], bar_count=20, frequency="1d")
    context.alphas = [alpha_1(data), alpha_2(data), alpha_3(data), alpha_4(data), alpha_5(data), alpha_6(data)]

# Run the backtest
start = pd.Timestamp("2021-01-01", tz="UTC")
end = pd.Timestamp("2023-01-01", tz="UTC")
initial_cash = 100000

run_algorithm(start=start,
              end=end,
              capital_base=initial_cash,
              initialize=initialize,
              before_trading_start=before_trading_start,
              data_frequency='daily',
              bundle='quantopian-quandl')


ValueError: no data for bundle 'quantopian-quandl' on or before 2024-09-07 18:21:06.958632+00:00
maybe you need to run: $ zipline ingest -b quantopian-quandl